
### Introduction

In the coursework for DATA 2060, we discussed a Naive Bayes algorithm which classified data represented as binary categorical features. We extend this algorithm to classify data represented as continuous features. This algorithm is called a Gaussian Naive Bayes algorithm. We follow the outline presented in the course textbook [1], Chapter 24.0 and 24.1.


#### Representation


To classify, we want to maximize $P[Y=y|\mathbf{X}=\mathbf{x}]$. Therefore we define our classifier and rewrite it using Bayes rule
and $$\text{h}_{\text{Bayes}}(x) = \underset{y}{\arg\max} \; P[Y=y|\mathbf{X}=\mathbf{x}] = \underset{y}{\arg\max} \; \frac{P[Y = y] P[\mathbf{X} = \mathbf{x} | Y = y]}{P[\mathbf{X}=\mathbf{x}]}.$$

Since $P[X = x]$ is constant for each class, this is equivalent to $$ \text{h}_{\text{Bayes}}(x) = \underset{y}{\arg\max} \; P[Y = y] P[\mathbf{X} = \mathbf{x} | Y = y]. $$

We make the "naive" assumption of Naive Bayes: we assume that conditioned on the label, the features are independent. Let $d$ be the number of features. We apply the "naive" assumption to have
$$P[\mathbf{X}=\mathbf{x}|Y=y]=\prod _{j=1}^d P[X_j=x_j|Y=y].$$
For Gaussian Naive Bayes, we assume that the features are continuous and normally distributed conditioned on each class, which implies that the features  take values in $\mathbb{R}$. Using these assumptions, the classifier learns distributions of the features conditioned on each class. In particular, these distributions are assumed to be normal, and thus, we can estimate the probability density functions for
$P[X_j=x_j | Y=y]$.

Let $\mu_{j,y}\in \mathbb{R}$ and $\sigma_{j,y}^2 \in \mathbb{R}$ be the mean and variance respectively for the distribution of feature $j$ when conditioned on $y$. Note $X_j=x_j | Y=y \sim N(\mu_{j,y},\sigma_{j,y}^2 )$ by the Gaussian assumption. Then
$$\begin{align*}P[\mathbf{X}=\mathbf{x}|Y=y]&= \prod _{j=1}^d P[X_j=x_j|Y=y] \quad \text{ by independence}\\
 &=\prod _{j=1}^d \frac{1}{\sqrt{2π}σ_{j,y}}e^{-\frac{1}{2}\frac{\left( x_j-\mu_{j,y}\right)^2}{σ_{j,y}^2}}\quad \text{ by the PDF of }X_j=x_j | Y=y\\
&= \text{exp}  \left(-\frac{1}{2} \sum _{j=1}^d \frac{\left( x_j-\mu^y _{j}\right)^2}{σ_j^2} \right)\left( \prod _{j=1}^d \frac{1}{\sqrt{2π}σ_j} \right)  \end{align*}$$
where the last step is given by products of exponentials.
Considering all features at once, let $\mu_y = (\mu_{1,y},\mu_{2,y},...,\mu_{d,y})$ be the mean vector for the features conditioned on label $y$.
Let $\Sigma_y$ be the covariance matrix of the features conditioned on label $y$. That is
$$\Sigma_y = \begin{pmatrix} σ_{1,y}^2& 0 & \cdots&\cdots &0\\
0& σ_{2,y}^2 & 0&&\vdots\\
\vdots&\ddots&\ddots&\ddots&\vdots\\
\vdots&&0&σ_{d-1,y}^2&0\\
0 &\cdots&\cdots&0&σ_{d,y}^2\end{pmatrix}$$
Note that since we are assuming indepencence of the features given the label, the covariance matrix of the features is simply a diagonal matrix with $\Sigma_{(j,j), y}=\sigma_{j, y}^2$. Under this form, we have $\text{det} \Sigma_y=\prod _{j=1}^d σ_{j,y}^2$ and that the inverse of the covariance matrix, $\Sigma_y^{-1}$, is a diagonal matrix with $\Sigma_{(j,j), y}= (\sigma_{j,y}^2)^{-1}$. Now we can rewrite the above expression as

$$P[\mathbf{X}=\mathbf{x}|Y=y] =  \frac{1}{\left( {2π} \right)^{\frac{d}{2}}\left( \text{det} \Sigma_y \right)^{\frac{1}{2}}} \; \text{exp}  \left(-\frac{1}{2} \left( x-\mu_y \right)^{T} \Sigma_y^{-1}\left( x-\mu_y \right) \right). $$




Lastly, we set the prior distribution $P[Y = y]$ to be the proportions of each class in the dataset. We apply Laplace smoothing to deal with the edge case of one class not appearing in the data which would make the prior distribution of the class zero. Thus, we have the prior distribution is calculates as follows
$$P[Y = y]= \frac{ \text{number of samples of class y}+1}{ \text{total number of samples}+\text{number of classes}}$$

We then can compute all expressions of
$$ P[Y = y] P[\mathbf{X} = \mathbf{x} | Y = y]$$
for each class $y \in Y$ for given example $x \in X$. We then classify the example $x$ based on
$$\text{h}_{\text{Bayes}}(x) = \underset{y}{\arg\max} P[Y = y] P[\mathbf{X} = \mathbf{x} | Y = y]$$
so that the assigned label has the highest probability $P[Y=y| \mathbf{X} =\mathbf{x}]$.

#### Loss and Optimizer

Let $S$ denote the training data. First, consider a sample $S_y$ denoting the subset of our training data of contining examples of just class $y$ such that $S_y\subseteq S$. We consider the $m$ examples of $S_y$ such that  $S_y= (\mathbf{x}_1,...,\mathbf{x}_m)$ for i.i.d. $\mathbf{x}_i \in \mathbb{R}^d$
for all $i = 1:m$ where $d $ is the number of features as before. We assume that $S$ was sampled from some overarching probablity distribution $P_{\theta_y}$ and we can then define the likelihood of this sample as
$$l(S_y;\theta_y) = \prod_{i=1}^m P_{\theta_y}(\mathbf{x}_i).$$
Moreover, we can define the log-likelihood  as
$$L(S_y;\theta_y) = \log\left(\prod_{i=1}^m P_{\theta_y}(\mathbf{x}_i)\right)= \sum_{i=1}^m \log(P_{\theta_y}(\mathbf{x}_i)).$$
For each $\mathbf{x}_i$, recall that we are assuming that the $j$th feature is normally distributed with mean $\mu_{j,y}$ and variance $\sigma_{j,y}$. In other words, we assume that the $j$th entry of $\mathbf{x}_i$, denoted as $x_{i,j}$, is distributed as $x_{i,j} \sim N(\mu_{j,y},\sigma_{j,y})$ for all $j=1:d$.  Then the overarching probability density distribution $P_{\theta_y}$ is parameterized by $\theta_y = (\boldsymbol{\mu}_y,\boldsymbol{\sigma}_y)$ where $\boldsymbol{\mu_y} = (\mu_{1,y},...,\mu_{d,y})$ and $\boldsymbol{\sigma_y} = (\sigma_{1,y},...,\sigma_{d,y})$. Since we are assuming that the features are independent, note that
$$P_{\theta_y}(x_i) = \prod_{j=1}^d P_{\theta_{j,y} }(x_i)$$
where $P_{\theta_{j,y} }$ is the distribution of the $j$th feature conditioned on $y$. Then, the log-likelihood can be written as
$$L(S_y;\theta_y) = \sum_{j=1}^d \sum_{i=1}^m \log(P_{\theta_j}(\mathbf{x}_i))= \sum_{j=1}^d L(S_y;\theta_{j,y} ).$$
Note that maximizing $L(S_y;\theta_y)$ is equivalent to maximizing $L(S_y;\theta_{j,y} )$ for each $j =1:d$. Since we assumed each feature was normally distributed, we then have
$$P_{\theta_{j,y} }(x_{i,j}) = \frac{1}{\sigma_{j,y}\sqrt{2\pi}}e^{\frac{- ( x_{i,j}-\mu_{j,y} )^2 }{2 \sigma{j,y} ^2} }.$$
Hence
$$\begin{align}
L(S_y;\theta_{j,y}) &= \sum_{i=1}^m\log\left(\frac{1}{\sigma_{j,y}\sqrt{2\pi}}e^{\frac{- ( x_{i,j}-\mu_{j,y} )^2 }{2 \sigma{j,y} ^2} }\right)\\
&= \sum_{i=1}^m\left(\frac{-(x_{i,j}-\mu_{j,y})^2}{2\sigma_{j,y}^2}+\log\left(\frac{1}{\sigma_{j,y}\sqrt{2\pi}}\right)\right)\\
&= -m\log(\sigma_{j,y}\sqrt{2\pi})-\frac{1}{2\sigma_{j,y}^2}\sum_{i=1}^m(x_{i,j}-\mu_{j,y})^2.
\end{align}$$
We are trying to maximize the log-likelihood using $\mathbf{\mu}$ and $\mathbf{\sigma}$. Taking the derivative with respect to $\mu_{j,y}$, we have
$$\frac{d}{d\mu_{j,y}}L(S_y;\theta_{j,y}) = \frac{1}{\sigma_{j,y}^2}\sum_{i=1}^m(x_{i,j}-\mu_{j,y}).$$
Then, with respect ot $\sigma_{j,y}$, we have
$$\frac{d}{d\sigma_{j,y}}L(S_y;\theta_{j,y}) =-\frac{m}{\sigma_{j,y}}+\frac{1}{\sigma_{j,y}^3}\sum_{i=1}^m(x_{i,j}-\mu_{j,y})^2.$$
Setting these derivatives equal to zero and solving for the respective parameter, we get
$$\begin{align}
\mu_{j,y}&= \frac{1}{m}\sum_{i=1}^m x_{i,j}\\
\sigma_{j,y} &=\sqrt{\frac{1}{m}\sum_{i=1}^m(x_{i,j}-\mu_{j,y})^2}.
\end{align}$$
Therefore, to maximize the likelihood of obtaining a sample in class $y$, we calculate $\mu_{j,y}$ and $\sigma_{j,y}$ to be the empirical mean and variance for each feature $j = 1:d$ conditioned on class $y$ to generate an estimate of $P_{\theta_{j,y} }$ for each feature. This choice then maximizes the log-likelihood $L(S_y;\theta_{j,y})$.

The loss of the parameter $\theta$ on a single observation $\mathbf{x}$ for this algorithm is the negative of the log-likelihood. This is called the log-loss and is defined as

$$ \ell(\theta, \mathbf{x}) = - \log (P_\theta(\mathbf{x}))$$

Therefore, minimizing the total log-loss is equivalent to maximizing the log-likelihood.

$$ \underset{\theta}{\operatorname{argmin}} \sum_{i=1}^m \ell(\theta, \mathbf{x}_i) = \underset{\theta}{\operatorname{argmax}} \sum_{i=1}^m \log(P_\theta(\mathbf{x}_i)). $$
We that letting $\theta = (\mathbf{\mu},\mathbf{\sigma})$ maximizes log-likelihood such that $\mathbf{\mu} =(\mu_1,...,\mu_d)$ and $\mathbf{\sigma}= (\sigma_1,...,\sigma_d)$  where each pair $(\mu_j,\sigma_j)$ is the empircal mean and variance for each feature $j=1:d$. Hence we minimize log loss the same choice of $\theta$.

#### The Algorithm
Using the theoretical we have given above, we proceed with our model below.

We create our class `GaussianNaiveBayes` where you initialize the object with the number of classes. You can then train the model using `train` which calculates the mean of each feature as well as the covariance matrix using `_get_params`. As described above, the model makes the naive assumption that assuming the features are independent. Therefore, the covariance matrix is the diagonal matrix with the feature variances on the diagonal and zeros elsewhere. We also calculate the label priors using Laplace smoothing to avoid issues with numerical stability.

We then can make predictions on a given sample using `predict` which predicts using equations FIX LATER. The model also calculates accuracy using mean 0-1 loss.

#### Model

In [ ]:
import numpy as np
import pandas as pd

class GaussianNaiveBayes(object):
    """ Gaussian Naive Bayes model

    @attrs:
        n_classes: the number of classes
        n_features: the number of features
        label_priors: a 1D NumPy array of the priors distribution
            (label_priors(i)=P(Y_i))
        means: a 2D array of size (n_classes,n_features) where Mu(i,j) is the
            mean of jth feature for the ith class
        covariance: a 3D array of size (n_classes,n_features,n_features)
            where covariance[i,:,:] = covariance of the features
            conditioned on the ith class
    """

    def __init__(self, n_classes):
        """ Initializes a NaiveBayes model with n_classes. """
        self.n_classes = n_classes
        self.n_features = None
        self.label_priors = None
        self.mean = None
        self.covariance = None

    def train(self, X_train, y_train):
        """ Trains the model, using maximum likelihood estimation.
        @params:
            X_train: a 2D (n_examples x n_attributes) numpy array
            y_train: a 1D (n_examples) numpy array
        @return:
            None
        """
        self.n_features = X_train.shape[1]
        # get means and covariances
        self._get_params(X_train, y_train)
        # compute prior dist of labels with laplace smoothing (so not zero)
        self.label_priors=np.zeros(self.n_classes)
        for i in range(self.n_classes):
          self.label_priors[i]= (sum(y_train==i)+1)/(len(y_train)+self.n_classes)
        pass

    def _get_params(self, X, Y):
        """ Calculated mean and covariance for each class.
        @params:
            X: a 2D (n_examples x n_attributes) numpy array
            Y: a 1D (n_examples) numpy array
        @return:
            None
        """
        self.mean = np.zeros((self.n_classes, X.shape[1]))
        self.covariance = np.zeros((self.n_classes, X.shape[1], X.shape[1]))
        for i in range(self.n_classes):
            X_i = X[Y == i]
            #empty case: no samples for class i NEED TO RETHINK THIS!!! maybe should fill with NaN. Then in the if the mean or Cov for a class contains NaN, return prob of that class as 0.
            if X_i.shape[0] == 0:
              self.mean[i, :] = np.nan
              self.covariance[i, :, :] = np.nan
              continue
            self.mean[i, :] = np.mean(X_i, axis=0)
            #calculate variance and place on diagonal (assume independence)
            var=np.zeros(self.n_features)
            for j in range(self.n_features):
              var[j]=np.var(X_i[:,j])
            self.covariance[i, :, :] = np.diag(var)
        pass

    def predict(self, inputs):
        """ Outputs a predicted label for each input in inputs.
        @params:
            inputs: a 2D NumPy array containing inputs
        @return:
            a 1D numpy array of predictions
        """
        labels = []
        for x in inputs:
          P_Y_X = []
          for i in range(self.n_classes):
            if (np.isnan(self.mean[i, :])).any() or (np.isnan(self.covariance[i, :, :])).any(): #if inputs[Y == i].shape[0] == 0: #np.nan in self.mean[i, :] or np.nan in self.covariance[i, :, :]:
              P_Y_X.append(0)
            else:
              coeff = 1/((2*np.pi)**(self.n_features/2)*np.sqrt(np.linalg.det(self.covariance[i,:,:])))
              P_X_Y  = coeff*np.exp(-.5*np.matmul(np.matmul((x-self.mean[i,:]).T,np.linalg.inv(self.covariance[i,:,:])),(x-self.mean[i,:])))
              P_Y_X.append(self.label_priors[i]*P_X_Y)
          labels.append(np.argmax(P_Y_X))
        return np.array(labels)


    def accuracy(self, X_test, y_test):
        """ Outputs the accuracy of the trained model on a given dataset (data).

        @params:
            X_test: a 2D numpy array of examples
            y_test: a 1D numpy array of labels
        @return:
            a float number indicating accuracy (between 0 and 1)
        """
        predictions = self.predict(X_test)
        return sum(predictions ==y_test)/len(y_test)



#### Check Model

Here we test the model. We use unit tests to check that each component is working correctly. We then run our model on the wine dataset from sklearn and compare it to sklearn's Guassian naive bayes algoriothm, GaussianNB.  

In [ ]:
import pytest
np.random.seed(0)


test_model1=GaussianNaiveBayes(2) #standard case
test_model2=GaussianNaiveBayes(3) #edge case: all one class
test_model3=GaussianNaiveBayes(3) #edge case: all equally distributed between classes

# Creates Test Data
#sample from gaussian- know linearly separable
x_1_0=np.random.normal(0,1,(10000,2))
x_1_1=np.random.normal(-20,1,(20000,2))
y1_0 = np.zeros(10000)
y1_1 = np.ones(20000)

x1= np.concatenate((x_1_0,x_1_1))
y1= np.concatenate((y1_0,y1_1))

x_test1 = np.concatenate((np.random.normal(0,1,(3,2)),np.random.normal(-20,1,(2,2))))
y_test1 = np.concatenate((np.zeros(3),np.ones(2)))
y_test1_true = np.concatenate((np.zeros(1),np.ones(4)))
#all one class
x2 = np.random.uniform(0,1,(10,3))
y2 = np.ones(10)
x_test2 = np.random.uniform(0,1,(4,3))
y_test2 = np.ones(4)
y_test2_true = np.array([1, 0, 1, 0])

#all equally distributed between each classe
x3 = np.array([[ 2.15,1.06],
[ 2.37, 2.55],
[ 1.32, 1.24],
[-2.95, 1.85],
[-2.78, 2.4 ],
[-2.77, 1.88],
[ 0.94, -3.42],
[ 1.36 ,-2.97],
[ 1.29, -3.17]])

y3 = np.array([0, 0, 0, 1, 1, 1, 2, 2, 2])
x_test3 = np.array([[ 1.68,  2.39],
 [ 1.45,  2.48],
 [ 1.51,  2.48],
 [-2.89,  2.09],
 [-3.26,  1.89],
 [-2.92,  2.34],
 [ 0.78, -2.76],
 [ 0.79, -3.21],
 [ 1.34, -2.71]])

y_test3 = np.array([0, 0, 0, 1, 1, 1, 2, 2, 2]) #what the model should predict
y_test3_true = np.array([0, 0, 0, 1, 1, 1, 2, 2, 1]) #"true" labels of data- used to check accuracy
# Test train
def check_train_dtype(model, x_train, y_train):
    assert isinstance(model.mean, np.ndarray)
    assert model.mean.ndim==2 and model.mean.shape==(model.n_classes, model.n_features)
    assert isinstance(model.covariance, np.ndarray)
    assert model.covariance.ndim==3 and model.covariance.shape==(model.n_classes, model.n_features, model.n_features)
    assert isinstance(model.label_priors, np.ndarray)
    assert model.label_priors.ndim==1 and model.label_priors.shape==(model.n_classes, )

def check_cov(x):
  var=np.zeros(x.shape[1])
  for j in range(x.shape[1]):
    var[j]=1/(x.shape[0])*np.sum((x[:,j]-np.mean(x[:,j]))**2)
  return np.diag(var)


test_model1.train(x1,y1)
check_train_dtype(test_model1, x1, y1)

assert (test_model1.mean == pytest.approx(np.array([[0.0,0.0],[-20.0,-20.0]]), abs=0.05)) #may have to loosen tol-using LLN
assert (test_model1.covariance == pytest.approx(np.array([np.eye(2),np.eye(2)]), abs= 0.05))#may have to loosen tol-using CLT
assert (test_model1.label_priors == pytest.approx(np.array([(10000+1)/(30000+2), (20000)/(30000+2)]),abs=0.01))
assert (test_model1.predict(x_test1) == y_test1).all()
assert (test_model1.accuracy(x_test1,y_test1_true) ==  pytest.approx(3/5, 0.01))


test_model2.train(x2,y2)
check_train_dtype(test_model2, x2, y2)
assert(np.isnan(test_model2.mean[0])).all()
assert(np.isnan(test_model2.mean[2])).all()
assert (test_model2.mean[1] == pytest.approx(np.array([np.mean(x2[:,0]),np.mean(x2[:,1]),np.mean(x2[:,2])]), abs=0.01))
assert(np.isnan(test_model2.covariance[0])).all()
assert(np.isnan(test_model2.covariance[2])).all()
assert (test_model2.covariance[1] == pytest.approx(check_cov(x2[y2==1]), 0.01))
assert (test_model2.label_priors == pytest.approx(np.array([(1)/(10+3), (10+1)/(10+3),(1)/(10+3)]), 0.01))
assert (test_model2.predict(x_test2) == y_test2).all()
assert (test_model2.accuracy(x_test2,y_test2_true) ==  pytest.approx(.5, 0.01))

test_model3.train(x3,y3)
check_train_dtype(test_model3, x3, y3)
assert (test_model3.mean == pytest.approx(np.array([[(2.15+2.37+1.32)/3,(1.06+2.55+1.24)/3],[(-2.95-2.78-2.77)/3,(1.85+2.4+1.88)/3],[(0.94+1.36+1.29)/3,(-3.42-2.97-3.17)/3]]), 0.01))
assert (test_model3.covariance == pytest.approx(np.array([check_cov(x3[y3==0]),check_cov(x3[y3==1]),check_cov(x3[y3==2])]), 0.01))
assert (test_model3.label_priors == pytest.approx(np.array([(3+1)/(9+3), (3+1)/(9+3), (3+1)/(9+3)]), 0.01))
assert (test_model3.predict(x_test3) == y_test3).all()
assert (test_model3.accuracy(x_test3,y_test3_true) ==  pytest.approx(8/9, 0.01))

In [ ]:
from sklearn.datasets import load_wine
from sklearn.naive_bayes import GaussianNB
import random
data = load_wine()
X = data.data
Y = data.target

# get training and testing data
indices = list(range(X.shape[0]))
random.shuffle(indices)

# 80% train and 20% test
train_index = int(np.ceil(X.shape[0]*0.7))

X_train = X[indices[:train_index]]
Y_train = Y[indices[:train_index]]
X_test = X[indices[train_index:X.shape[0]]]
Y_test = Y[indices[train_index:X.shape[0]]]

# train our model
our_wine_model = GaussianNaiveBayes(3)
our_wine_model.train(X_train,Y_train)
our_predictions = our_wine_model.predict(X_test)

sk_wine_model = GaussianNB()
sk_wine_model.class_count_ = 3
sk_wine_model.fit(X_train, Y_train)
sk_predictions = sk_wine_model.predict(X_test)

diff = (our_predictions!=sk_predictions)

print(f"Our accuracy: {our_wine_model.accuracy(X_test, Y_test)}")
print(f"Sklearn accuracy: {sk_wine_model.score(X_test,Y_test)}")
print(f"There are {np.sum(diff)} samples classified differently")



NameError: name 'np' is not defined

We compare our models predictions with the prediction of the gaussian naive bayes model in sklearn. Specifically, we compare the results of the model the "wine data set". This dataset is made up of different Italians wines from three different "cultivars"- ie made from different vinyards with different grapes. The features are the amount of 13 different components/ingredients in the wines. Hence, based off of these 13 continuous features we classify the wines as from one of the three cultivars.
The data is made up of 59, 71, and 48 samples from each cultivar- 178 samples in total.
Our model and the sklearn model have the exact same predictions on this dataset.

## References (remove numbers and alphebetize once everything is cited above)

[1] Shalev-Shwartz, S., and Ben-David, S. (2014). *Understanding machine learning: From theory to algorithms.* 1st edn. New York: Cambridge university press.

[2] Aeberhard, S., Coomans, D., and de Vel, O. (1994). 'The performance of statistical pattern recognition methods in high dimensional settings,' *Proc. IEEE Signal Process. Workshop Higher Order Statist* (pp. 14-16).

[3] scikit-learn developers. (2024) *load_wine* [Online]. Available at: https://scikit-learn.org/1.5/modules/generated/sklearn.datasets.load_wine.html (Accessed: 4 December 2024)

[4] scikit-learn developers. (2024) *GaussianNB* [Online]. Available at: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html (Accessed: 4 December 2024)